In [1]:
%cd ../..
# %pip install .
import time
from os import environ
# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50, 
#         with viewAngle 135 deg
# ped = Pedestrian on roadsOrIntersections,
#         with regionContainedIn roadRegion,
#         facing Range(-180, 180) deg

# require abs(relative heading of ped from ego) > 70 deg

/home/eecs/chanwutk/Documents/apperception


In [2]:
if "AP_PORT" not in environ:
    environ["AP_PORT"] = str(input('port'))
# README command uses port=25432

In [3]:
from apperception.world import empty_world
from apperception.database import database
from apperception.utils import F
from apperception.predicate import objects, camera, lit
database.connection

<connection object at 0x7f3c9aa677c0; dsn: 'user=docker password=xxx dbname=mobilitydb host=localhost port=25442', closed: 0>

In [4]:
name = 'ScenicWorld' # world name
world = empty_world(name=name)

o = objects[0]
c = camera
world = world.filter(
    F.like(o.type, lit('human.pedestrian%')) &
    F.contained(c.ego, F.road_segment('road')) &
    (F.contained_margin(o.bbox@c.time, F.road_segment('road'), lit(0.50)) | F.contained(o.trans@c.time, F.road_segment('road'))) &
    F.angle_excluding(F.facing_relative(o.traj@c.time, c.ego), lit(-70), lit(70)) &
    F.angle_between(F.facing_relative(c.ego, F.road_direction(c.ego, c.ego)), lit(-15), lit(15)) &
    (F.distance(c.camAbs, o.traj@c.time) < lit(50)) &
    (F.view_angle(o.trans@c.time, c.cam) < lit(35))
)

start = time.time()
# keys = world.get_traj_key()
id_time_camId_filename = world.get_id_time_camId_filename(num_joined_tables=1)

end = time.time()
print(format(end-start))

execute: filter
execute: get_id_time_camId_filename
get_id_time_camId_filename 
        SELECT t0.itemId, cameras.timestamp, cameras.cameraId, cameras.filename
        FROM (SELECT * FROM item_general_trajectory) as t0
        
        JOIN Cameras USING (cameraId)
        WHERE (t0.objectType LIKE 'human.pedestrian%' AND contained(egoTranslation,roadSegment('road')) AND (containedMargin(objectBBox(t0.itemId, timestamp),roadSegment('road'),0.5) OR contained(valueAtTimestamp(t0.translations,timestamp),roadSegment('road'))) AND angleExcluding(facingRelative((valueAtTimestamp(t0.itemHeadings,timestamp))::real, egoHeading),-70,70) AND angleBetween(facingRelative(egoHeading, roadDirection(egoTranslation,egoHeading)),-15,15) AND (distance(cameraTranslationAbs,valueAtTimestamp(t0.trajCentroids,timestamp))<50) AND (viewAngle(valueAtTimestamp(t0.translations,timestamp),cameraHeading,cameraTranslation)<35))
        
done execute node
Result length: 83
10.204400539398193


In [5]:
resultImages = dict() # maping from image -> (itemId, timestamp, camId) that it was found at
for result in id_time_camId_filename:
    itemId, timestamp, camId, filename = result
    filename = filename.split("/")[-1] # use split so that prefix path is not included in filename
    resultImages[filename] = (itemId, timestamp, camId)

In [6]:
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import os
import random

experiment_data_dir =  "data/nuscenes/experiment_data"
# experiment_data_dir = '/work/apperception/data/nuScenes/full-dataset-v1.0/Trainval/experiment_data'
def show_images(images, sample=None):
    if sample is not None:
        images = [i for i in images]
        random.shuffle(images)
        images = images[:sample]
    
    plt.figure(figsize=(60,30))
    columns = 3

    for i, image in enumerate(images):
        print("image", image)
        img = mpimg.imread(os.path.join(experiment_data_dir, image))
        print("loaded")
        plt.subplot(len(images) // columns + 1, columns, i + 1)
        plt.imshow(img)

In [7]:
# show_images(list(resultImages.keys()), sample=10)

In [8]:
expected_imgs = [
    'n008-2018-05-21-11-06-59-0400__CAM_FRONT__1526915471412465.jpg',
    'n008-2018-07-27-12-07-38-0400__CAM_FRONT__1532707917112404.jpg',
    'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385153162404.jpg',
    'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385153662404.jpg',
    'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385154162404.jpg',
    'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385154662404.jpg',
    'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385155162404.jpg',
    'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385158662404.jpg',
    'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385159162404.jpg',
    'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385159662404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656805162404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656805662415.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656806162404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656806612404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656807162404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656807662404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656808162404.jpg',
    'n008-2018-08-30-15-52-26-0400__CAM_FRONT__1535659404362404.jpg',
    'n008-2018-08-30-15-52-26-0400__CAM_FRONT__1535659404762404.jpg',
    'n008-2018-08-30-15-52-26-0400__CAM_FRONT__1535659405262404.jpg',
    'n008-2018-08-30-15-52-26-0400__CAM_FRONT__1535659405762404.jpg',
    'n008-2018-08-30-15-52-26-0400__CAM_FRONT__1535659406262404.jpg',
    'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535728830362404.jpg',
    'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729326412404.jpg',
    'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729326912404.jpg',
    'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729327412404.jpg',
    'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729327912404.jpg',
    'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729328412404.jpg',
    'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729328912404.jpg',
    'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729329412404.jpg',
    'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729329912795.jpg',
    'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729330362404.jpg',
    'n008-2018-08-31-11-37-23-0400__CAM_FRONT__1535730293412404.jpg',
    'n008-2018-08-31-11-56-46-0400__CAM_FRONT__1535731236162404.jpg',
    'n008-2018-08-31-11-56-46-0400__CAM_FRONT__1535731236662404.jpg',
    'n008-2018-08-31-11-56-46-0400__CAM_FRONT__1535731237112404.jpg',
    'n008-2018-08-31-11-56-46-0400__CAM_FRONT__1535731237612404.jpg',
    'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293291162404.jpg',
    'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293291662404.jpg',
    'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293292162404.jpg',
    'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293292662404.jpg',
    'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293293162404.jpg',
    'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293293662404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299143862404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299144362404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299144862404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299145362404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299145862404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299146362404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299146862404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299147362404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299147862404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299148362404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299148862404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299149412404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299229362404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299239112404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299240112404.jpg',
]

In [9]:
# show_images(expected_imgs, sample=10)

In [10]:
missing = 0
missing_images = []
for expected_img in expected_imgs:
    if expected_img not in resultImages.keys():
        missing += 1
        missing_images.append(expected_img)
print("Percentage of expected results missing from query: ", missing, "/", len(expected_imgs), "=", 100 * missing / len(expected_imgs), "%")
show_images(missing_images, sample=10)

Percentage of expected results missing from query:  42 / 58 = 72.41379310344827 %
image n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729328412404.jpg


FileNotFoundError: [Errno 2] No such file or directory: 'data/nuscenes/experiment_data/n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729328412404.jpg'

<Figure size 4320x2160 with 0 Axes>

In [11]:
extra = 0
extra_images = {}
for img in resultImages.keys():
    if img not in expected_imgs:
        extra += 1
        extra_images[img] = resultImages[img]
print("Percentage of images in query but not in expected results: ", extra, "/", len(resultImages.keys()), "=", 100 * extra / len(resultImages.keys()), "%")
show_images(extra_images.keys(), sample=10)

Percentage of images in query but not in expected results:  33 / 49 = 67.34693877551021 %
image n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537292946912404.jpg


FileNotFoundError: [Errno 2] No such file or directory: 'data/nuscenes/experiment_data/n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537292946912404.jpg'

<Figure size 4320x2160 with 0 Axes>

In [12]:
######## Getting info about tthe extra images ########

In [13]:
from apperception.database import database
for img in extra_images:
    itemId, timestamp, camId = extra_images[img]
    prefix = "samples/CAM_FRONT/"
    query = f"SELECT viewAngle(table_0.translations, Cameras.cameraHeading, Cameras.cameraTranslationAbs, Cameras.timestamp), \'Cameras: \', Cameras.filename, roadDirection(Cameras.egoTranslation, Cameras.timestamp, Cameras.egoHeading), Cameras.egoHeading, \'Ego Translation: \', ST_X(Cameras.egoTranslation), ST_Y(Cameras.egoTranslation), ST_Z(Cameras.egoTranslation), \'Cam Translation: \', ST_X(Cameras.cameraTranslationAbs), ST_Y(Cameras.cameraTranslationAbs), ST_Z(Cameras.cameraTranslationAbs), \'Pedestrian: \',table_0.itemId, getX(table_0.trajCentroids, Cameras.timestamp), getY(table_0.trajCentroids, Cameras.timestamp), ST_Z(valueAtTimestamp(table_0.trajCentroids, Cameras.timestamp)), valueAtTimestamp(table_0.itemHeadings, Cameras.timestamp), roadDirection(CAST(getX(table_0.trajCentroids, Cameras.timestamp) AS real), CAST(getY(table_0.trajCentroids, Cameras.timestamp) AS real), -9.99) FROM Item_General_Trajectory AS table_0, Cameras" + \
                f" WHERE Cameras.filename = \'{prefix + img}\' AND table_0.itemId = \'{itemId}\'"
    result = database._execute_query(query)
    print(result)

UndefinedFunction: function getx(tgeompoint, timestamp with time zone) does not exist
LINE 1: ...raTranslationAbs), 'Pedestrian: ',table_0.itemId, getX(table...
                                                             ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.


In [14]:
######## Seeing what predicates are not being satisfied for missing images ########

In [15]:
# We assume for now that there is no problem with the commented out predicates, since they are relativley simple
predicates = [
    # "(table_0.objectType LIKE 'human.pedestrian%')",
    "contained(Cameras.egoTranslation, roadSegment('road'))",
    "(contained(table_0.translations, roadSegment('road'), Cameras.timestamp) OR contained(table_0.translations, roadSegment('intersection'), Cameras.timestamp))",
    "(containedMargin(objectBBox(table_0.itemId, Cameras.timestamp), roadSegment('road'), 0.60, Cameras.timestamp) OR containedMargin(objectBBox(table_0.itemId, Cameras.timestamp), roadSegment('intersection'), 0.5, Cameras.timestamp))",
    "angleExcluding(facingRelative(table_0.itemHeadings, Cameras.egoHeading, Cameras.timestamp), (-70), 70)",
    "angleBetween(facingRelative(Cameras.egoHeading, roadDirection(Cameras.egoTranslation, Cameras.timestamp, Cameras.egoHeading), Cameras.timestamp), (-15), 15)",
    # "(distance(Cameras.egoTranslation, table_0.trajCentroids, Cameras.timestamp)<50)",
    # "(viewAngle(table_0.trajCentroids, Cameras.egoHeading, Cameras.egoTranslation, Cameras.timestamp)<135)"
    
]

In [16]:
from apperception.database import database
preds_missed = {}
for img in missing_images:    
    prefix = "samples/CAM_FRONT/"
    query = f"SELECT COUNT(*) FROM Item_General_Trajectory AS table_0, Cameras" + \
                f" WHERE Cameras.filename = \'{prefix + img}\' AND (table_0.objectType LIKE 'human.pedestrian%') AND (distance(Cameras.cameraTranslationAbs, table_0.trajCentroids, Cameras.timestamp)<50) AND (viewAngle(table_0.trajCentroids, Cameras.egoHeading, Cameras.cameraTranslationAbs, Cameras.timestamp)<135)" 
    database.cursor.execute(query)
    result = database.cursor.fetchall()
    num_peds = result[0][0]
    # For now we only take a look at images with one pedestrian (so that we know that that one pedestrian is the one that is supposed to satisfy the condition)
    # (if still experiencing problems after fixing the errors with the images with only one pedestrian, we can expand)
    if num_peds != -1:
        # print("------", img, "------")
        for predicate in predicates:
            prefix = "samples/CAM_FRONT/"
            query = f"SELECT true FROM Item_General_Trajectory AS table_0, Cameras" + \
                        f" WHERE Cameras.filename = \'{prefix + img}\' AND (table_0.objectType LIKE 'human.pedestrian%') AND (distance(Cameras.egoTranslation, table_0.trajCentroids, Cameras.timestamp)<50) AND (viewAngle(table_0.trajCentroids, Cameras.egoHeading, Cameras.egoTranslation, Cameras.timestamp)<35) AND " + \
                        predicate 
            result = database._execute_query(query)
            print(result)
            # if the predictae was not satisfied
            if len(result) == 0: 
                if predicate not in preds_missed:
                    preds_missed[predicate] = 0
                preds_missed[predicate] += 1
                print(predicate)
                query = f"SELECT {predicate}, Cameras.filename, table_0.itemId, objectBBox(table_0.itemId, Cameras.timestamp), ST_Area(objectBBox(table_0.itemId, Cameras.timestamp)::box3d::geometry) FROM Item_General_Trajectory AS table_0, Cameras" + \
                        f" WHERE Cameras.filename = \'{prefix + img}\' AND (table_0.objectType LIKE 'human.pedestrian%') AND (distance(Cameras.egoTranslation, table_0.trajCentroids, Cameras.timestamp)<50) AND (viewAngle(table_0.trajCentroids, Cameras.egoHeading, Cameras.egoTranslation, Cameras.timestamp)<35)" 
                result = database._execute_query(query)
                print(result)

            

[(True,)]
[(True,)]
[(True,)]
[(True,)]
[(True,)]
[(True,), (True,)]
[]
(contained(table_0.translations, roadSegment('road'), Cameras.timestamp) OR contained(table_0.translations, roadSegment('intersection'), Cameras.timestamp))
[(False, 'samples/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385153162404.jpg', 'c24e096fcf4c4e55b74892bddc982eaa', 'STBOX ZT((2153.495244698027,919.030510704958,0.020500000000000185,2018-08-27 08:52:33.162404+00),(2153.6974703240962,919.7931546560656,0.020500000000000185,2018-08-27 08:52:33.162404+00))', 0.15422615048069907), (False, 'samples/CAM_FRONT/n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385153162404.jpg', '69c08f8de9eb4af1ade1a6f85f9421fd', 'STBOX ZT((2154.2170801681277,918.6168837443242,0.15599999999999992,2018-08-27 08:52:33.162404+00),(2154.4577577781233,919.4550115914018,0.15599999999999992,2018-08-27 08:52:33.162404+00))', 0.20171860710538148)]
[]
(containedMargin(objectBBox(table_0.itemId, Cameras.timestamp), roadSegment('road'), 

In [ ]:
for pred in preds_missed:
    print(pred, preds_missed[pred])